In [69]:
import os
import requests
import pandas as pd
from tqdm.notebook import trange, tqdm
from os.path import join
import shutil

In [83]:
root_folder = "C:/Users/Simon/Data/BOON DR CAMP PROJECT 2019-2021/"
df = pd.read_excel(join(root_folder, "BOON DR CAMP PROJECT 2019-2021_preparation for R 1.1.xlsx"))
df = df.drop(df.columns[-29:],axis=1)
print(df.columns)
print(f'Dataset length: {len(df)}')
#df.head()

Index(['DATE OF EXAMINATION', 'CAMP LOCATION', 'PATIENT ID', 'GENDER',
       'AGE (YEARS)', 'EDUCATION', 'BP', 'RBS', 'HEIGHT (cm)', 'WEIGHT (kg)',
       'HYPERTENSION', 'DIABETES', 'FAMILY HISTORY OF DIABETES',
       'DURATION OF DIABETES', 'DIABETIC MEDICATION',
       'DIABETIC MEDICATION GROUP',
       'SYSTEMIC ABNORMALITY \nOPTIONS- NIL, CARDIAC, NEPHROPATHY, STROKE, OTHERS',
       'RE VISUAL ACUITY', 'LE VISUAL ACUITY', 'RE LENS STATUS',
       'LE LENS STATUS', 'RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'RE PAXOS DISC CENTRED IMAGE UPLOAD', 'RE PAXOS TEMPORAL IMAGE UPLOAD',
       'RE PAXOS STQ IMAGE UPLOAD', 'RE PAXOS ITQ IMAGE UPLOAD',
       'RE PAXOS SNQ IMAGE UPLOAD', 'RE PAXOS INQ IMAGE UPLOAD',
       'RE PAXOS OTHER IMAGES UPLOAD',
       'LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'LE PAXOS DISC CENTRED IMAGE UPLOAD', 'LE PAXOS TEMPORAL IMAGE UPLOAD',
       'LE PAXOS STQ IMAGE UPLOAD', 'LE PAXOS ITQ IMAGE UPLOAD',
       'LE PAXOS SNQ IMAGE UPLOAD',

In [84]:
# Remove empty rows
'''df_data = df.loc[1:, ['PATIENT ID','RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'RE PAXOS DISC CENTRED IMAGE UPLOAD', 'RE PAXOS TEMPORAL IMAGE UPLOAD',
       'RE PAXOS STQ IMAGE UPLOAD', 'RE PAXOS ITQ IMAGE UPLOAD',
       'RE PAXOS SNQ IMAGE UPLOAD', 'RE PAXOS INQ IMAGE UPLOAD',
       'RE PAXOS OTHER IMAGES UPLOAD',
       'LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'LE PAXOS DISC CENTRED IMAGE UPLOAD', 'LE PAXOS TEMPORAL IMAGE UPLOAD',
       'LE PAXOS STQ IMAGE UPLOAD', 'LE PAXOS ITQ IMAGE UPLOAD',
       'LE PAXOS SNQ IMAGE UPLOAD', 'LE PAXOS INQ IMAGE UPLOAD',
       'LE PAXOS OTHER IMAGES UPLOAD']]'''

df_data = df.loc[1:, :]
df_data = df_data.drop(df_data[df_data['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'] == '[]'].index)
df_data = df_data.drop(df_data[df_data['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'] == 'null'].index)
df_data = df_data[(df_data['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'].notna() | df_data['LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'].notna())]
#df_data.tail()
print(f'Cleaned dataset length: {len(df_data)}')
df_data.to_excel(join(root_folder, "BOON DR CAMP PROJECT 2019-2021 cleaned 1.1.xlsx"), index=False)

Cleaned dataset length: 223


In [41]:
# attempt to download data from aws
base_url = 'https://teleopticsapp.s3.ap-south-1.amazonaws.com/'
base_folder = 'aws'
failed_attempts = 0

if not os.path.exists(base_folder):
    os.mkdir(base_folder)

for i, row in tqdm(df_data.iterrows(), total=len(df_data)):
    if row['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'][0] != '[':
        # skip locally saved files
        continue
    url_list = eval(row['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'])
    for url in url_list:
        res = requests.get(url)
        img_data = res.content
        file_name = url.replace(base_url, '')
        file_name = file_name.replace('/', '_')
        if not res.ok:
            print(f'Failed download from {url}')
            failed_attempts += 1
        else: 
            with open(os.path.join(base_folder, file_name), 'wb') as h:
                h.write(img_data)
        
    

  0%|          | 0/219 [00:00<?, ?it/s]

Failed download from https://teleopticsapp.s3.ap-south-1.amazonaws.com/CZ20211022003/RE/POST/REPOSTMC_20211024_237.jpg


In [85]:
# Check whether tele-opthmologists and optometrist grading is the same
OPT_RE = 'OPTOMETRIST \nRE :  DR GRADE - PAXOSCOPE POST-DILATION'
OPT_LE = 'OPTOMETRIST LE :  DR GRADE - PAXOSCOPE POST-DILATION'
OPH_RE = 'Ophthalmologist\nRE TELEGRADING' 
OPH_LE = 'Ophthalmologist\nLE TELEGRADING'

df_data = df_data.replace('CANT GRADE ', '')
df_data = df_data.replace('CANNOT COMMENT', '')

for i, row in tqdm(df_data.iterrows(), total=len(df_data)):
    if pd.isna(row[OPT_RE]) or pd.isna(row[OPH_RE]):
        pass
    elif not row[OPT_RE].split(' ')[0].lower() == row[OPH_RE].split(' ')[0].lower():
        print(f"Conflicting grading for {i}, {row['PATIENT ID']}, right eye: {row[OPT_RE]}/{row[OPH_RE]}")
    if pd.isna(row[OPT_LE]) or pd.isna(row[OPH_LE]):
        pass
    elif not row[OPT_LE].split(' ')[0].lower() == row[OPH_LE].split(' ')[0].lower():
        print(f"Conflicting grading for {i}, {row['PATIENT ID']}, left eye: {row[OPT_LE]}/{row[OPH_LE]}")

# 81, 160421003 -> NO DR
# 71, 1311200002 -> NO DR
# 62, 280220003RR -> MODERATE NPDR
# 60, 280220001PR -> MILD NPDR
# 56, 220220001PR -> MODERATE NPDR
# 46, 020220001KA -> MODERATE NPDR
# 45, 02022002PA -> MILD NPDR
# 44, 120120008SK -> MILD NPDR
# 34, DR270120001A -> MILD NPDR
# 27, 191220002BSR -> NO DR
# 26, 191220001RM -> MILD NPDR
# 23, 191215011 -> CANNOT COMMENT
# 19, 191215006GJ -> MILD NPDR
# 18, 191215005VS -> NO DR/MILD NPDR
# 17, 191215004Z -> RE: CANNOT COMMENT, LE: MODERATE NPDR

  0%|          | 0/223 [00:00<?, ?it/s]

Conflicting grading for 17, 191215004Z, right eye: /MODERATE NPDR
Conflicting grading for 17, 191215004Z, left eye: SEVERE NPDR/MODERATE NPDR
Conflicting grading for 18, 191215005VS, right eye: NO DR/MILD NPDR
Conflicting grading for 19, 191215006GJ, right eye: MODERATE NPDR/MILD NPDR
Conflicting grading for 20, 191215008AA, right eye: NO DR/
Conflicting grading for 23, 191215011, left eye: /NO DR
Conflicting grading for 26, 191220001RM, left eye: Moderate NPDR/MILD NPDR
Conflicting grading for 27, 191220002BSR, right eye: NO DR/
Conflicting grading for 34, DR270120001A, right eye: mild NPDR/NO DR
Conflicting grading for 44, 120120008SK, right eye: NO DR/MILD NPDR
Conflicting grading for 45, 02022002PA, left eye: NO DR/MILD NPDR
Conflicting grading for 46, 020220001KA, right eye: MILD NPDR/MODERATE NPDR
Conflicting grading for 56, 220220001PR, right eye: SEVERE NPDR/MODERATE NPDR
Conflicting grading for 56, 220220001PR, left eye: SEVERE NPDR/MODERATE NPDR
Conflicting grading for 60, 28

In [86]:
# Move images to examination folder according to excel file
dir_name = join(root_folder, "sorted_by_eye_pp/")
lookup_folder = 'images_pp' # for non-processed images'SANKARA UKB DR STUDY_Images'
right_eye_cols = ['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD','RE PAXOS DISC CENTRED IMAGE UPLOAD', 'RE PAXOS TEMPORAL IMAGE UPLOAD',
       'RE PAXOS STQ IMAGE UPLOAD', 'RE PAXOS ITQ IMAGE UPLOAD', 'RE PAXOS SNQ IMAGE UPLOAD', 'RE PAXOS INQ IMAGE UPLOAD',
       'RE PAXOS OTHER IMAGES UPLOAD']
left_eye_cols = ['LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD', 'LE PAXOS DISC CENTRED IMAGE UPLOAD', 'LE PAXOS TEMPORAL IMAGE UPLOAD',
       'LE PAXOS STQ IMAGE UPLOAD', 'LE PAXOS ITQ IMAGE UPLOAD', 'LE PAXOS SNQ IMAGE UPLOAD', 'LE PAXOS INQ IMAGE UPLOAD',
       'LE PAXOS OTHER IMAGES UPLOAD']

if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
os.mkdir(dir_name)
for i, row in tqdm(df_data.iterrows(), total=len(df_data)):
    pid = row['PATIENT ID']
    if i > 82:
        continue
    dir_le, dir_re = join(dir_name, str(pid) + '_LE'), join(dir_name, str(pid) + '_RE')
    os.mkdir(dir_le)
    os.mkdir(dir_re)
    
    for j, col in enumerate(right_eye_cols):
        if not pd.isna(row[col]): 
            row[col] = join(lookup_folder, row[col].split('/')[1])
        if not pd.isna(row[col]) and os.path.exists(join(root_folder, row[col])):
            shutil.copy(join(root_folder, row[col]), join(dir_re, os.path.basename(row[col])))
    for j, col in enumerate(left_eye_cols):
        if not pd.isna(row[col]):
            row[col] = join(lookup_folder, row[col].split('/')[1])
        if not pd.isna(row[col]) and os.path.exists(join(root_folder, row[col])):
            shutil.copy(join(root_folder, row[col]), join(dir_le, os.path.basename(row[col])))

created_folders = os.listdir(dir_name)
deletions = 0
for folder in created_folders:
    if len(os.listdir(join(dir_name, folder))) == 0:
        os.rmdir(join(dir_name, folder))
        deletions += 1
print(f'{deletions} emtpy folders were removed after population')

  0%|          | 0/223 [00:00<?, ?it/s]

6 emtpy folders were removed after population


In [88]:
# translate filenames to Paxos nomentclature: ([A-Z])(\d){3}[RL](\d)?
camp_prefix = 'X'
sorted_dir = join(root_folder, "sorted_by_eye_pp/")
renamed_dir = join(root_folder, "renamed_and_sorted_pp")
created_folders = os.listdir(sorted_dir)
os.mkdir(renamed_dir)
for i, folder in enumerate(created_folders):
    suffix = folder.split('_')[1][0]
    exam_id = f'{camp_prefix}{i:03d}{suffix}'
    os.mkdir(join(renamed_dir, exam_id))
    for j, img in enumerate(os.listdir(join(sorted_dir, folder))):
        shutil.copy(join(sorted_dir, folder, img), join(renamed_dir, exam_id, f'{exam_id}_{j:02d}{os.path.splitext(img)[1]}'))
        shutil.copy(join(sorted_dir, folder, img), join(renamed_dir, f'{exam_id}_{j:02d}{os.path.splitext(img)[1]}'))
    with open(join(renamed_dir, exam_id, 'legacy.txt'), "w") as f:
        f.write(folder)

In [ ]:
# 1200 files
# 1171 lose 30 files through cropping
# 823 lose 340 files which are not mentioned in the excel file